In [1]:
import sys
import time
import torch
import copy
from torch.backends import cudnn
from matplotlib import colors
import cv2
import numpy as np
import os
import statistics
from skimage import io, draw
from yolox.data.datasets import COCO_CLASSES
from yolox.utils import fuse_model, get_model_info, postprocess, vis
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from loguru import logger
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
import yolo_detect as yd

#마스크 생성해주기 
def mask(zone,w,h):  
    mask = np.zeros((h, w), dtype=np.uint8)
    mask.fill(255)

    
    zxs = [z for idx, z in enumerate(zone) if idx % 2 == 0]
    zys = [z for idx, z in enumerate(zone) if idx % 2 != 0]
    zxs = np.array(zxs)
    zys = np.array(zys)
    x, y = draw.polygon(zys, zxs)
    mask[x, y] = 128
    return mask


#바운딩박스 그려주는 함수         
def vis(img, boxes, scores, conf = 0.5):
    for i in range(len(boxes)):
        box = boxes[i]
        score = scores[i]
        if score < conf:
            continue
        x0 = int(box[0])
        y0 = int(box[1])
        x1 = int(box[2])
        y1 = int(box[3])

        color = (0, 0, 255)
        text = '{}:{:.1f}%'.format(f'NO', score * 100)
        txt_color = (255, 255, 255)
        font = cv2.FONT_HERSHEY_SIMPLEX

        txt_size = cv2.getTextSize(text, font, 0.6, 1)[0]
        cv2.rectangle(img, (x0, y0), (x1, y1), color, 2)

        txt_bk_color = (0, 0, 0)
        cv2.rectangle(
            img,
            (x0, y0 + 1),
            (x0 + txt_size[0] + 1, y0 + int(1.5*txt_size[1])),
            txt_bk_color,
            -1
        )
        cv2.putText(img, text, (x0, y0 + txt_size[1]), font, 0.4, txt_color, thickness=1)

    return img

#클래스 (차량, 오토바이 등) 분류해주고 박스 원래 사이즈로 돌려놓기         
def visual(output,frame,img_info,cls_conf):
    det = yd.YoloDetector_red_lights() 
    img = frame


    ratio = img_info["ratio"]
    
    #아무런 output이 나오지 않을경우 
    if output is None:
        return None, None, None, None, None, None
    output = output.cpu()
    bboxes = output[:, 0:4]
    # preprocessing: resize
    bboxes /= ratio
    cls = output[:, 6] #현재 이미지에서 잡힌 객체들의 번호 차 = 2번 , 신호등 = 9 번 
    scores = output[:, 4] * output[:, 5] #잡힌 객체들의 신뢰도 수준 
    
    person_bboxes = []
    person_scores = []
    person_draw_bboxes = []
    person_draw_scores = []
    
    moto_bboxes = []
    moto_scores = []
    #박스 원래 사이즈로 돌려놓기 
    x_min = img_info["x_min"]
    y_min = img_info["y_min"]
    for i in range(len(bboxes)):
        box = bboxes[i]
        cls_id = int(cls[i])
        score = scores[i]
        
        #오리지날 이미지 사이즈대로 박스 원복 시키기 
        x0 = int(box[0]) + x_min
        y0 = int(box[1]) + y_min
        x1 = int(box[2]) + x_min
        y1 = int(box[3]) + y_min
        
        #이미지 사이즈 넘어가는것들은 값 맞춰주기 
        if x0 >= img.shape[1]:
            x0 = img.shape[1] - 1
        if x1 >= img.shape[1]:
            x1 = img.shape[1] - 1
        if y0 >= img.shape[0]:
            y0 = img.shape[0] - 1
        if y1 >= img.shape[0]:
            y1 = img.shape[0] - 1
        
        #사람 발 밑 좌표 두개 
        three = y1, x0 
        four = y1, x1
        
        #횡단보도에 있는 사람 바운딩 박스만 적재 하기

        if cls_id == 0 and cls_conf < score and (mask[three] == 128 and mask[four] == 128) :
            person_bboxes.append([x0,y0,x1,y1])
            person_scores.append(score)
            person_draw_bboxes.append([x0,y0,x1,y1])
            person_draw_scores.append(score)
        
        #횡단보도에 있는 오토바이 바운딩 박스만 적재 하기 
        elif cls_id == 3 and cls_conf < score and (mask[three] == 128 and mask[four] == 128) :
            moto_bboxes.append([x0,y0,x1,y1])
            moto_scores.append(score)


    #output은 나왔지만 차량에 해당 하는 output이 없을경우 (사람만 포착이 되었을경우)     
    if len(person_bboxes) == 0 and len(moto_bboxes) == 0  :
        return None, None, None, None , None, None

    
    return person_bboxes, person_scores , moto_bboxes,moto_scores, person_draw_bboxes, person_draw_scores

def del_moto(person_bboxes, person_scores, moto_bboxes,moto_scores,person_draw_bboxes, person_draw_scores):
    #오토바이 리스트를 돌면서 현재 사람좌표가 오토바이 좌표 내에 있는지 확인을 하고 있을경우 bbox 리스트에서 제거를하는 for문 
    
    global before_moto
    #사람 박스 하나씩 돌기 
    for i in range(len(person_bboxes)):
        print('i',i)
        p_box = person_bboxes[i]
        print('p_box',p_box)
        score = person_scores[i]
        x0 = int(p_box[0]) 
        y0 = int(p_box[1]) 
        x1 = int(p_box[2]) 
        y1 = int(p_box[3])
        print('before_moto',before_moto)
#         print('moto_bboxes',moto_bboxes)
        #오토바이가 검출이 되었을경우 
        if not len(moto_bboxes) == 0 : 
            for idx in moto_bboxes:
                #오토바이 박스 안에 사람이 있을겨웅 if문 통과 
                if (idx[0] -50 <= x0 <= idx[2] +50  and idx[1] -50  <= y1 <= idx[3] + 50 ) and ( idx[0] -50 <= x1 <= idx[2] +50  and idx[1] -50 <= y1 <= idx[3] + 50 ):
                    #오토바이 안에 사람이 없으면 패스 
                    if not p_box in person_draw_bboxes : 
                        pass 
                    #오토바이 안에 사람이 있으면 그 사람을 리스트에서 제거 해주기 
                    else : 
                        person_draw_bboxes.remove(p_box)
                        person_draw_scores.remove(score)
                else :
                    pass
            #이전 박스랑 비교하기 
            for idx in before_moto:
                #
                if (idx[0] -50<= x0 <= idx[2] +50  and idx[1] -50  <= y1 <= idx[3] + 50 ) and ( idx[0] -50 <= x1 <= idx[2] +50  and idx[1] -50 <= y1 <= idx[3] + 50 ):
                    if not p_box in person_draw_bboxes : 
                        pass 
                    else : 
                        person_draw_bboxes.remove(p_box)
                        person_draw_scores.remove(score)
                else :
                    pass
        #오토바이 검출이 안되었을경우            
        else : 
            for idx in before_moto:
                if (idx[0] -50<= x0 <= idx[2] +50  and idx[1] -50  <= y1 <= idx[3] + 50 ) and ( idx[0] -50 <= x1 <= idx[2] +50  and idx[1] -50 <= y1 <= idx[3] + 50 ):
                    print('before_moto 오토방구 탄사람 ')
                    person_draw_bboxes.remove(p_box)
                    person_draw_scores.remove(score)
                else :
                    pass
    #오토바이 검출이 되었을때 before_moto 업데이트 시켜주기 
    if not len(moto_bboxes) == 0 :
        before_moto = []
        before_moto = moto_bboxes
    else :
        pass 
#     print('before_moto',before_moto)
    return person_draw_bboxes, person_draw_scores
                




#비디오 읽고, 전체적인 코드 실행 
def proc(video_path,output_file):
    det = yd.YoloDetector_red_lights() #detector 소환      
    det.load() #모델 load
    cap = cv2.VideoCapture(video_path)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  # float
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float
    fps = cap.get(cv2.CAP_PROP_FPS)
    print('fps',fps)
    save_path = output_file
    logger.info(f"video save_path is {save_path}")
    vid_writer = cv2.VideoWriter(
        save_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (int(width), int(height))
    )

    cnt = 0
    if cap.isOpened() :    
        while True:
            cnt+=1
            #원본이미지 = frame 
            ret, frame = cap.read()
            if ret == False :
                break
            if cnt < 0:
                continue
            #프레임수 특정값 이상일떄는 break 할때 
#             if cnt > 10:
#                 break
            #현재 프레임
            num_frame = str(round(cap.get(cv2.CAP_PROP_POS_FRAMES)))
            print('frame : ',num_frame)
            #detect 완료후 결과 받기
            outputs, img_info = det.inference(frame,num_frame,img_zone,cut_rate_x, cut_rate_y, camera_number)

            if outputs[0] == None :
                result_frame = frame 
                continue
            
            #특정 클래스만 받아오기 
            person_bboxes, person_scores , moto_bboxes,moto_scores, person_draw_bboxes, person_draw_scores = visual(outputs[0], frame,img_info,det.confthre)
            
            if person_bboxes == None and moto_bboxes == None : 
                result_frame = frame
                continue 
            #오토바이 제거하기 
            if not len(moto_bboxes) == 0 :
                draw_people_box, draw_people_score = del_moto(person_bboxes, person_scores , moto_bboxes,moto_scores, person_draw_bboxes, person_draw_scores)
                if not len(draw_people_box) == 0 :
                    result_frame = vis(frame, draw_people_box, draw_people_score,conf = 0.5)
                else : result_frame = frame 
            else : result_frame = vis(frame, person_bboxes, person_scores,conf = 0.5)
            vid_writer.write(result_frame)
            ch = cv2.waitKey(1)
            if ch == 27 or ch == ord("q") or ch == ord("Q"):
                break

                
                

#시나리오 번호 (조건절 추가해서 시나리오 번호 설정 하면 zone 선택하도록 진행 )
camera_number = 3061 

#전체 이미지 자를 영역 
# img_zone = [253,400,680,449,1437,569,1565,68,1479,0,1078,0,649,151]  # 시나리오 4번
# img_zone = [213, 290, 847, 227, 1105, 214, 1412, 214, 1480, 302, 1654, 427, 423, 875, 213, 290]  # 3065
img_zone = [1229,250,1724,279,1666,470,1508,1055,747,781,307,651,837,392,1017,319,1229,250]  # 3061
# img_zone = [417, 278, 781, 364, 1483, 606, 1271, 1079, 540, 1079, 209, 635, 79, 350]  # 3082
# img_zone = [905, 244, 1313, 270, 1558, 403, 1841, 602, 1597, 663, 1055, 809, 950, 400, 905, 244] #3044


if img[y1][x1][2] >= 100 or img[y2][x2][2] >= 100 or img[y3][386][2] >= 100:


cut_rate_x = 0.3
cut_rate_y = 0.3

#############################################마스크###################################################### 
w, h = 1920, 1080
# mask_zone = [253, 400, 680, 449, 1437, 569, 1565, 68, 1479, 0, 1078, 0, 649, 151]  # 시나리오 4번
# mask_zone = [213, 290, 847, 227, 1105, 214, 1412, 214, 1480, 302, 1654, 427, 423, 875, 213, 290]  # 3065
mask_zone = [1229, 250, 1724, 279, 1666, 470, 1508, 1055, 747, 781, 307, 651, 837, 392, 1017, 319, 1229, 250]  # 3061
# mask_zone = [417, 278, 781, 364, 1483, 606, 1271, 1079, 540, 1079, 209, 635, 79, 350] #3082
# mask_zone = [905, 244, 1313, 270, 1558, 403, 1841, 602, 1597, 663, 1055, 809, 950, 400, 905, 244]  # 3044
#속도 느려서 아예 만들어 줘야함 
mask = mask(mask_zone,w,h)
############################################################################################################



#전 프레임 오토바이 적재하기 
before_moto = []

input_list = [
    './input_3061/1.mp4',
#     './input/2.mp4',
#     './input/3.mp4',
#     './input/4.mp4',
#     './input/5.mp4',
#     './input/6.mp4',
#     './input/7.mp4',
#     './input/8.mp4',
#     './input/9.mp4',
#     './input/10.mp4',
#     './input/11.mp4',
#     './input/12.mp4',
]
output_list = [
    './output_3044/3.mp4',
#     './output/2.mp4',
#     './output/3.mp4',
#     './output/4.mp4',
#     './output/5.mp4',
#     './output/6.mp4',
#     './output/7.mp4',
#     './output/8.mp4',
#     './output/9.mp4',
#     './output/10.mp4',
#     './output/11.mp4',
#     './output/12.mp4',
]


for idx, input_path in enumerate(input_list):
    print(f"{input_path} start!!")
    proc(input_list[idx],output_list[idx])
    print(f'####################{input_list[idx]} complete#########################')

./input_3061/1.mp4 start!!


2022-04-12 03:22:57.128 | INFO     | yolo_detect:load:191 - Model Summary: Params: 99.07M, Gflops: 281.93
2022-04-12 03:23:00.948 | INFO     | __main__:proc:211 - video save_path is ./output_3044/3.mp4


fps 31.0
frame :  1
frame :  2
frame :  3
frame :  4
frame :  5
frame :  6
frame :  7
frame :  8
frame :  9
frame :  10
frame :  11
frame :  12
frame :  13
frame :  14
frame :  15
frame :  16
frame :  17
frame :  18
frame :  19
frame :  20
frame :  21
frame :  22
frame :  23
frame :  24
frame :  25
frame :  26
frame :  27
frame :  28
frame :  29
frame :  30
frame :  31
frame :  32
frame :  33
frame :  34
frame :  35
frame :  36
frame :  37
frame :  38
frame :  39
frame :  40
frame :  41
frame :  42
frame :  43
frame :  44
frame :  45
frame :  46
frame :  47
frame :  48
frame :  49
frame :  50
frame :  51
frame :  52
frame :  53
frame :  54
frame :  55
frame :  56
frame :  57
frame :  58
frame :  59
frame :  60
frame :  61
frame :  62
frame :  63
frame :  64
frame :  65
frame :  66
frame :  67
i 0
p_box [955, 237, 999, 369]
before_moto []
i 1
p_box [788, 407, 863, 572]
before_moto []
frame :  68
frame :  69
i 0
p_box [960, 233, 1004, 361]
before_moto [[823, 440, 929, 621]]
i 1
p_box [80

frame :  125
i 0
p_box [1231, 453, 1330, 646]
before_moto [[1166, 532, 1392, 677], [788, 451, 904, 613]]
frame :  126
i 0
p_box [1233, 457, 1337, 651]
before_moto [[1174, 515, 1399, 678], [788, 465, 911, 589]]
frame :  127
i 0
p_box [1242, 453, 1349, 651]
before_moto [[1179, 490, 1407, 678], [788, 449, 914, 600]]
i 1
p_box [803, 406, 898, 577]
before_moto [[1179, 490, 1407, 678], [788, 449, 914, 600]]
frame :  128
i 0
p_box [1252, 455, 1352, 652]
before_moto [[1189, 493, 1414, 681]]
i 1
p_box [813, 409, 909, 577]
before_moto [[1189, 493, 1414, 681]]
frame :  129
i 0
p_box [1257, 454, 1360, 654]
before_moto [[1194, 486, 1421, 683]]
i 1
p_box [820, 413, 913, 586]
before_moto [[1194, 486, 1421, 683]]
frame :  130
i 0
p_box [1262, 456, 1364, 653]
before_moto [[1198, 498, 1428, 685], [809, 476, 946, 622]]
i 1
p_box [830, 417, 927, 588]
before_moto [[1198, 498, 1428, 685], [809, 476, 946, 622]]
frame :  131
i 0
p_box [1270, 456, 1372, 655]
before_moto [[1203, 510, 1434, 687]]
frame :  132
i 

i 0
p_box [1358, 474, 1454, 669]
before_moto [[1271, 490, 1536, 701], [1207, 705, 1412, 927]]
i 1
p_box [1260, 613, 1400, 876]
before_moto [[1271, 490, 1536, 701], [1207, 705, 1412, 927]]
frame :  174
i 0
p_box [1272, 622, 1419, 941]
before_moto [[1281, 490, 1536, 702], [1232, 722, 1433, 938]]
i 1
p_box [1357, 475, 1451, 669]
before_moto [[1281, 490, 1536, 702], [1232, 722, 1433, 938]]
frame :  175
i 0
p_box [1285, 626, 1424, 937]
before_moto [[1289, 496, 1535, 700]]
i 1
p_box [1357, 474, 1445, 667]
before_moto [[1289, 496, 1535, 700]]
frame :  176
i 0
p_box [1297, 635, 1438, 932]
before_moto [[1293, 493, 1535, 700]]
i 1
p_box [1357, 474, 1444, 667]
before_moto [[1293, 493, 1535, 700]]
frame :  177
i 0
p_box [1311, 649, 1458, 938]
before_moto [[1288, 492, 1535, 700]]
i 1
p_box [1358, 471, 1464, 667]
before_moto [[1288, 492, 1535, 700]]
frame :  178
i 0
p_box [1358, 471, 1446, 671]
before_moto [[1287, 495, 1536, 701]]
frame :  179
i 0
p_box [1358, 474, 1450, 669]
before_moto [[1282, 491

frame :  321
frame :  322
frame :  323
frame :  324
frame :  325
frame :  326
frame :  327
frame :  328
frame :  329
frame :  330
frame :  331
frame :  332
frame :  333
frame :  334
frame :  335
frame :  336
frame :  337
frame :  338
frame :  339
frame :  340
frame :  341
frame :  342
frame :  343
frame :  344
frame :  345
frame :  346
frame :  347
frame :  348
frame :  349
frame :  350
frame :  351
####################./input_3061/1.mp4 complete#########################


In [11]:
asdf = {
    1 : {
        'helmet' : []
    }
    
}

In [12]:
if 1 in asdf:
    print ('exist')

exist


In [13]:
if 'helmet' in asdf[1]:
    print ('h exist')

h exist


In [14]:
asdf[1]['helmet'].append('qkjelqjwe')

In [15]:
asdf

{1: {'helmet': ['qkjelqjwe']}}